In [1]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "L

### Προσοχή

Μη διαγράψετε τα # insert your code here σχόλια, καθώς βοηθούν στη διόρθωση. Συμπληρώστε τον κώδικά σας μετά από τα σχόλια αυτά.

# Μέρος Α: Fine-tune a pretrained model

Τα γλωσσικά μοντέλα αποτελούνται από δύο στάδια εκπαίδευσης:
1. **Pre-training σε μεγάλα unlabelled datasets**:

  Το pre-training είναι υπολογιστικά πολύ ακριβό και γι αυτό στην πράξη δε το χρησιμοποιούμε όταν θέλουμε να τρέξουμε ένα μοντέλο σε ένα καινούργιο dataset. Μπορούμε να σκεφτούμε το pre-training ως τη διαδικασία εκμάθησης γλωσσικών κανόνων κι εννοιών, οι οποίες στη συνέχεια μπορούν να χρησιμοποιηθούν για διάφορους σκοπούς.

2. **Fine-tuning σε μικρότερα labelled datasets**:
  
     Το fine-tuning πρακτικά εκμεταλλεύεται τις ιδιότητες του transfer learning προκειμένου να μεταφέρουμε τη 'γνώση' που έχει αποθηκευθεί στο γλωσσικό μοντέλο κατά τη διάρκεια του pre-training σε συγκεκριμένα task. Κάθε task εξυπηρετείται μέσω στοχευμένων datasets. Για παράδειγμα, κάποια datasets αναφέρονται στην ταξινόμηση κειμένων σε κατηγιορίες (text classification), άλλα datasets περιέχουν ερωτήσεις οι οποίες πρέπει να απαντηθούν (question answering) κι άλλα πολλά.

Κάποια κλασικά tasks της επεξεργασίας φυσικής γλώσσας είναι τα ακόλουθα:
- Text classification
- Question answering
- Natural language inference
- Fill mask
- Semantic similarity

Περισσότερες πληροφορίες μπορείτε να βρείτε στον ακόλουθο σύνδεσμο στο domain Natural Language Processing: https://huggingface.co/models

Στο πρώτο κομμάτι της παρούσας εργαστηριακής άσκησης, θα χρησιμοποιήσουμε το pre-training fine-tuning σενάριο για να ταξινομήσουμε reviews.

## Pipelines

Με τη χρήση του **text-classification pipeline** μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης.

Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες: **[entailment/neutral/contradiction]**.

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι **γραμματικά ορθό (acceptable) ή όχι (unacceptable)**:

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια.
Το  Yelp κατασκευάστηκε θεωρώντας τα αστέρια 1 και 2 αρνητικά και τα 3 και 4 θετικά.  Η αρνητική πολικότητα ανήκει στην κατηγορία 1 και η θετική στην κατηγορία 2. Τα reviews αυτά χωρίζονται σε αυτές τις κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.



In [ ]:
from datasets import load_dataset

# insert your code here

reviews = load_dataset('fancyzhx/yelp_polarity')
print(reviews)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.93k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 38000
    })
})


Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνουμε να διατηρήσετε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγξτε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρήστε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [ ]:
import collections
print("Classes:", reviews['train'].unique('label'))

label_counts = collections.Counter(reviews['train']['label'])
print("Label Counts:", label_counts)

Classes: [0, 1]
Label Counts: Counter({0: 280000, 1: 280000})


In [ ]:
# insert your code here
from datasets import concatenate_datasets

def subset_data(split):
  class_0_samples = reviews[split].filter(lambda example: example['label'] == 0).select(range(150))
  class_1_samples = reviews[split].filter(lambda example: example['label'] == 1).select(range(150))
  return concatenate_datasets([class_0_samples, class_1_samples]).shuffle(seed=42)

In [ ]:
train_dataset = subset_data('train')
eval_dataset = subset_data('test')

Filter:   0%|          | 0/560000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/560000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/38000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/38000 [00:00<?, ? examples/s]

In [ ]:
print("Training Label Counts", collections.Counter(train_dataset['label']))
print("Evaluation Label Counts", collections.Counter(eval_dataset['label']))

Training Label Counts Counter({1: 150, 0: 150})
Evaluation Label Counts Counter({1: 150, 0: 150})


# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα.

Η διαδικασία αυτή επιτελείται μέσω των **Tokenizers**, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, **αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer**. Επιλέγοντας τη μέθοδο **from_pretrained** λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλείστε να επιτελέσετε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής σας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξετε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώστε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

In [ ]:
# insert your code here
from transformers import AutoTokenizer, RobertaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Σας παρέχουμε τη συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξατε. Εφαρμόστε το τόσο στο train, όσο και στο test set.

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# insert your code here
train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Τυπώνοντας το train ή το test set, θα δείτε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθείτε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

In [ ]:
print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 300
})
Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 300
})


## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Προτείνουμε να τη χρησιμοποιήσετε ως εναλλακτική του να γράψετε το δικό σας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [ ]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορείτε να πειραματιστείτε κατά τη διαδικασία fine-tuning.


Καλείστε να πειραματιστείτε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσετε optimizer και scheduler για το fine-tuning. Προτείνουμε να εκτελέσετε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Θα μας δώσετε σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσατε και το accuracy που πετύχατε στην τελευταία εποχή.

2. Βάσει των πειραματισμών, πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξατε;  Σχολιάστε και αναλύστε.

In [ ]:
from transformers import TrainingArguments, Trainer
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

def train_evaluate_model(learning_rate, batch_size, weight_decay):
  model = RobertaForSequenceClassification.from_pretrained("roberta-base")  # default num labels = 2 (also model definition must be included as function gets called multiple times)

  args = TrainingArguments(
            output_dir="test_trainer",
            eval_strategy="epoch",
            #logging_strategy="epoch",
            per_device_train_batch_size=batch_size,
            learning_rate=learning_rate,
            num_train_epochs=3,
            weight_decay=weight_decay,
        )

  # insert your code here
  # optimizer
  optimizer = AdamW(
    model.parameters(),
    lr=args.learning_rate,
    weight_decay=args.weight_decay,
    eps=1e-8,
  )

  # scheduler
  scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=0, # basically no warm up more suitable for fine tuning
      num_training_steps=(300 // args.per_device_train_batch_size) * args.num_train_epochs,
  )

  trainer = Trainer(
      model=model,
      args=args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      compute_metrics=compute_metrics,
      optimizers=(optimizer, scheduler),
  )

  trained_model=trainer.train()

  return

Στη συνέχεια, ρυθμίστε (fine-tune) το μοντέλο σας καλώντας το [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
train_evaluate_model(learning_rate=5e-5, batch_size=16, weight_decay=0.01)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.281164,0.940000
2,No log,0.184761,0.946667
3,No log,0.240232,0.946667


In [ ]:
train_evaluate_model(learning_rate=1e-4, batch_size=16, weight_decay=0.01)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.327671,0.873333
2,No log,0.405010,0.860000
3,No log,0.256862,0.940000


In [ ]:
train_evaluate_model(learning_rate=5e-5, batch_size=32, weight_decay=0.01)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.579584,0.723333
2,No log,0.203969,0.940000
3,No log,0.188078,0.940000


In [ ]:
train_evaluate_model(learning_rate=1e-4, batch_size=16, weight_decay=0.1)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.376934,0.893333
2,No log,0.361385,0.853333
3,No log,0.313057,0.930000


In [ ]:
train_evaluate_model(learning_rate=2e-5, batch_size=8, weight_decay=0.0)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.221471,0.930000
2,No log,0.327875,0.900000
3,No log,0.242271,0.943333


In [ ]:
train_evaluate_model(learning_rate=1e-4, batch_size=32, weight_decay=0.1)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.200034,0.933333
2,No log,0.280777,0.930000
3,No log,0.273446,0.943333


In [ ]:
train_evaluate_model(learning_rate=1e-4, batch_size=32, weight_decay=0.01)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.687044,0.500000
2,No log,0.275290,0.933333
3,No log,0.204123,0.943333


In [ ]:
train_evaluate_model(learning_rate=5e-5, batch_size=16, weight_decay=0.001)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.311758,0.926667
2,No log,0.323070,0.920000
3,No log,0.248170,0.940000


In [ ]:
train_evaluate_model(learning_rate=1e-5, batch_size=8, weight_decay=0.001)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.675654,0.520000
2,No log,0.315067,0.936667
3,No log,0.197105,0.943333


In [ ]:
train_evaluate_model(learning_rate=1e-5, batch_size=8, weight_decay=0)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.675654,0.520000
2,No log,0.315067,0.936667
3,No log,0.197105,0.943333


####Hyperparameter tuning

| Learning Rate | Batch Size | Weight Decay | Accuracy |
|---------------|------------|--------------|----------|
| 5e-5          | 16         | 0.01         | 0.94667     |
| 1e-4          | 16         | 0.01        | 0.94000     |
| 5e-5          | 32          | 0.01       | 0.94000     |
| 1e-4          | 16         | 0.1        | 0.93000     |
| 2e-5          | 8         | 0.0       | 0.94333     |
| 1e-4          | 32         | 0.1       | 0.94333     |
| 1e-4          | 32         | 0.01       | 0.94333     |
| 5e-5          | 16         | 0.001       | 0.94000     |
| 1e-5          | 8         | 0.001       | 0.94333     |
| 1e-5          | 8         | 0.0       | 0.94333     |

####Ανάλυση Αποτελεσμάτων

Στη περίπτωση μας όπου έχουμε είδη ένα δυνατό pre trained model και μικρό dataset (300 samples), η χρήση μεγάλου **learning rate** (>1e-4) δε συνιστάται σε fine tuning διότι μπορεί το μοντέλο να "ξεχάσει" αυτά που έμαθε (εφόσον μεγάλα weight updates μπορούν να αλλοιώσουν τα βάρη που είχε είδη το roberta). Οπότε μείναμε στο range [1e-5 , 1e-4] που είναι ιδανικό εφόσον το μοντέλο μαθαίνει (αυξάνεται το accuracy).

Λόγου του μικρού dataset size μας περιοριζόμαστε σε **batch sizes** μέχρι το 32 ωστε να αποφύγουμε κακά accuracies. Μικρά batch sizes (<=8) προσφέρουν noise στο μοντέλο μας και συμπεριφέρονται ως ένα reguralization method οδηγώντας σε καλή γενίκευση του μοντέλου μας χωρίς να χρειαστεί να χρησημοποιήσουμε weight decay και επίσης είναι memory efficient. Απο την άλλη μεγαλύτερα batch sizes είναι πιο αποδοτικά (πιο γρήγορη εκπαίδευση) με χρήση gpu. Γενικά πάντως όλα τα batchsizes [8,32] που δοκιμάσαμε πετυχαίνουν παρόμοιο accuracy στο τελευταίο epoch.

Η χρήση του **weight decay** ως reguralization method είναι πολύ σημαντική ώστε να αποφύγουμε το overfitting και το μοντέλο μας να πετύχει καλύτερη γενίκευση. Γενικά η standard τιμή που πέτυχε το καλύτερο αποτέλεσμα είναι το 0.01 ενω το 0.1 θεωρείται πιο aggressive και θα ήταν περισσότερο χρήσημο μόνο όταν παρατηρούσαμε overfitting σε μεγαλύτερο βαθμό.

Συμπερασματικά, όπως βλέπουμε και απο το προηγούμενο πίνακα , οι διαφορές στο accuracy δεν είναι μεγαλές και αυτό οφείλετα ότι περιοριστίκαμε σε ranges για όλες τις προηγούμενες υπερπαραμέτρους που είναι κατάλληλα για fine tuning σε 3 εποχές. Τα μεγάλα accuracies που πετύχαμε επιβαιβαιώνουν τη χρησημότητα του Transfer learning, όπου πολύ αποδοτικά μπορούμε να έχουμε ένα μοντέλο που πετυχαίνει καλά αποτελέσματα στο dataset μας.





# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks

Στο κομμάτι αυτό της εργασίας δε χρειάζεται να πραγματοποιήσετε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή λογική (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [ ]:
# # insert your code here (load dataset)
from datasets import load_dataset

dataset = load_dataset('piqa')
subset = dataset['train'].shuffle(seed=42).select(range(100))
print(subset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.41k [00:00<?, ?B/s]

piqa.py:   0%|          | 0.00/5.36k [00:00<?, ?B/s]

The repository for piqa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/piqa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/16113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3084 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1838 [00:00<?, ? examples/s]

Dataset({
    features: ['goal', 'sol1', 'sol2', 'label'],
    num_rows: 100
})


In [ ]:
# print the first 5 rows of subset
features = ['goal', 'sol1', 'sol2', 'label']
for i in range(5):
    for key in features:
        print(f"{key}: {subset[key][i]}")

goal: Do cardio exercise without running.
sol1: Use a jump rope for 15 minutes.
sol2: Run around a chair for 15 minutes.
label: 0
goal: To add a promo code to your Uber order
sol1: Open the Uber app.  Tap the History button.  Tap Payment.  Tap Add Promo/Gift Code.  Type in your PROMO CODE.  Tap ADD.
sol2: Open the Uber app.  Tap the menu button.  Tap Payment.  Tap Add Promo/Gift Code.  Type in your PROMO CODE.  Tap ADD.
label: 1
goal: wet towel
sol1: hold  marker stains if washable marker gets on it
sol2: hold  water if washable marker gets on it
label: 1
goal: Bracing the base pf your platform with 2 x 4.
sol1: To take any wobble out of or base we will add two braces. First we will cut a 45-degree angle out of the top ends on each 2 x 4. This is so that you can attach the 2 x 4 to the inside of the platform. We will form a V with your 2 x 4 so that they overlap at a straight part of tree but also cleanly abut onto the inside of the platform. Attach the top of the bracing to the platfo

In [ ]:
subset[:2]

{'goal': ['Do cardio exercise without running.',
  'To add a promo code to your Uber order'],
 'sol1': ['Use a jump rope for 15 minutes.',
  'Open the Uber app.  Tap the History button.  Tap Payment.  Tap Add Promo/Gift Code.  Type in your PROMO CODE.  Tap ADD.'],
 'sol2': ['Run around a chair for 15 minutes.',
  'Open the Uber app.  Tap the menu button.  Tap Payment.  Tap Add Promo/Gift Code.  Type in your PROMO CODE.  Tap ADD.'],
 'label': [0, 1]}

Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Καλείστε λοιπόν να καταγράψετε το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης χρησιμοποιήστε τουλάχιστον 5 κατάλληλα μοντέλα.

⚠️⚠️ Η προσέγγιση της χρήσης multiple choice classification head χωρίς προηγούμενη εκπαίδευση (fine-tuning) – δηλαδή, χρησιμοποιώντας το μοντέλο απευθείας σε inference mode, όπως φαίνεται να συμβουλεύει η εκφώνηση σε αυτή την περίπτωση – αναμενόμενα δεν αποδίδει καλά. Αυτό συμβαίνει διότι ουσιαστικά το head λειτουργεί με τυχαία βάρη (random weights).

Μια πιθανή λύση θα ήταν η φόρτωση μοντέλων που έχουν ήδη υποστεί fine-tuning στο dataset PIQA. Ωστόσο, ο αριθμός τέτοιων μοντέλων είναι περιορισμένος. Επιπλέον, πολλά από αυτά αντιμετωπίζουν προβλήματα, όπως ελλιπή αρχεία στα ρεπο τους στο Hugging Face, γεγονός που καθιστά τη φόρτωσή τους μη τετριμμένη (AutoModel και AutoTokenizer δε λειτουργούν).

👁️👁️Για αυτό έχοντας υπόψην τα προηγούμενα θα πρέπει να ανάγουμε το πρόβλημα Piqa σε ένα κατάλληλο nlp task. Το nlp task που επιλέχθηκε μετά απο δοκιμές αλλα και απο θεωρητική μελέτη είναι το **Natural Language Inference** (ΝLI). Ουσιαστικά η αναγωγή γίνεται ως εξής, θεωρούμε το goal ως το premise και τα 2 solutions ως τις υποθέσεις. H υπόθεση (solution) με το μεγαλύτερο entailment score είναι αυτή που θα επιλεχθεί.

In [11]:
# insert your code here (models)
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm

# Define NLI models to evaluate (these are specifically trained for NLI)
MODELS = [
    "tasksource/deberta-small-long-nli",
    "tasksource/deberta-base-long-nli",
    "sileod/deberta-v3-base-tasksource-nli",
    "tasksource/ModernBERT-base-nli",
    "sileod/deberta-v3-large-tasksource-nli"
]

In [ ]:
# insert your code here (function for ending prediction)
def evaluate_model(model_name):
    print(f"\nEvaluating: {model_name}")

    # Load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name).eval() # no need since its in evaluation by default, also nli is basically classification for 3 classes

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    correct = 0
    total = 0

    for example in tqdm(subset, desc="Processing examples"):
        premise = example['goal']
        hypotheses = [example['sol1'], example['sol2']]
        label = example['label']  # 0 or 1

        # Tokenize premise-hypothesis pairs
        inputs = [tokenizer(premise, hyp, return_tensors='pt')
                 for hyp in hypotheses]

        # Get entailment scores (usually index 0 for "entailment" in NLI models)
        with torch.no_grad():
            scores = []
            for input in inputs:
                output = model(input['input_ids'].to(device),
                             attention_mask=input['attention_mask'].to(device))
                logits = output.logits # example tensor([[-2.7232,  1.9883,  0.2172]]), which are scores for entailment (the one we care about), contradiction and neutral, higher score means more confidence
                entail_score = logits[0][0].item()  # index may vary by model
                scores.append(entail_score)

        # Predict the solution with higher entailment score
        predicted = 0 if scores[0] > scores[1] else 1
        if predicted == label:
            correct += 1
        total += 1

    accuracy = correct / total
    print()
    print(f"Accuracy: {accuracy:.2f} ({correct}/{total})")
    return accuracy

In [ ]:
# Evaluate all models
results = {}
for model in MODELS:
    results[model] = evaluate_model(model)


Evaluating: tasksource/deberta-small-long-nli


Processing examples: 100%|██████████| 100/100 [00:43<00:00,  2.31it/s]



Accuracy: 0.67 (67/100)

Evaluating: tasksource/deberta-base-long-nli


Processing examples: 100%|██████████| 100/100 [01:29<00:00,  1.11it/s]



Accuracy: 0.72 (72/100)

Evaluating: sileod/deberta-v3-base-tasksource-nli


...


Accuracy: 0.83 (83/100)

Evaluating: tasksource/ModernBERT-base-nli


Processing examples: 100%|██████████| 100/100 [00:50<00:00,  1.99it/s]



Accuracy: 0.48 (48/100)

Evaluating: sileod/deberta-v3-large-tasksource-nli


Processing examples: 100%|██████████| 100/100 [04:57<00:00,  2.98s/it]



Accuracy: 0.84 (84/100)


In [ ]:
print("\nFinal Results:")
for model, accuracy in results.items():
    print(f"{model}: {accuracy:.2f}")


Final Results:
tasksource/deberta-small-long-nli: 0.67
tasksource/deberta-base-long-nli: 0.72
sileod/deberta-v3-base-tasksource-nli: 0.83
tasksource/ModernBERT-base-nli: 0.48
sileod/deberta-v3-large-tasksource-nli: 0.84


Το **καλύτερο accuracy** που πετύχαμε είναι **84%** που είναι αρκετά ικανοποιητικό.

## B2. Truthful QA

### Sentence Transformers

Οι **sentence transformers** χρησιμοποιούνται για να δημιουργήσουν **embeddings προτάσεων**, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Για παράδειγμα, εκτελέστε το ακόλουθο κελί, το οποίο δίνει μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). Μπορείτε ακόμα να δοκιμάσετε να εκτελέσετε το ακόλουθο κελί για διαφορετικές προτάσεις της επιλογής σας, που μπορεί να είναι όμοιες ή πολύ διαφορετικές μεταξύ τους, και να παρατηρήσετε τις αλλαγές τιμών του cosine similarity.

In [4]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

np.float32(0.40488452)

Για τη συνέχεια της άσκησης, καλείστε να επιλέξετε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή.

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορείτε να θεωρήσετε τα ίδια μοντέλα και να τα επεκτείνετε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Καλείστε λοιπόν να γράψετε μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξατε στο ερώτημα Β1.


👉🏽👉🏽  Το τρέξιμο κάθε μοντέλου απο το προηγούμενο ερώτημα 6 φορές για κάθε sentence transformer είναι μη αποδοτικό, ειδικά όταν πάρουμε υπόψην το μέγεθος τους. Έτσι για κάθε μοντέλο, θα χρησημοποιήσουμε 2 sentence transformers ώστε συνολικά να έχουμε δοκιμάσει τουλάχιστον 6 sentence transformes μοντέλα.

In [7]:
# insert your code here (load dataset)
from datasets import load_dataset
dataset = load_dataset('truthful_qa', 'generation')
subset = dataset['validation'].filter(lambda x: len(x['correct_answers']) >= 2).shuffle(seed=42).select(range(100))
print(subset)

README.md:   0%|          | 0.00/9.59k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/223k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

Filter:   0%|          | 0/817 [00:00<?, ? examples/s]

Dataset({
    features: ['type', 'category', 'question', 'best_answer', 'correct_answers', 'incorrect_answers', 'source'],
    num_rows: 100
})


In [8]:
# insert your code here (load models for semantic similarity and QA)
SEMANTIC_MODELS = [
    'sentence-transformers/all-MiniLM-L6-v2',
    'sentence-transformers/multi-qa-mpnet-base-dot-v1',
    'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    'sentence-transformers/all-distilroberta-v1',
    'sentence-transformers/stsb-roberta-large',
    'sentence-transformers/gtr-t5-large'
]

# Load all semantic models once
semantic_models = {name: SentenceTransformer(name) for name in SEMANTIC_MODELS}

# split them into pairs
model_names = list(semantic_models.keys())
models = list(semantic_models.values())
pair_list = []
for i in range(len(model_names)-1):
    pair_list.append(((model_names[i], models[i]), (model_names[i+1], models[i+1])))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

In [12]:
# insert your code here (function for optimal correct answers & semantic similarity)
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
import torch
from tqdm import tqdm

def evaluate_with_semantic_similarity(dataset, similarity_threshold=0.95):
    results = {}

    for idx, model_name in enumerate(MODELS):
        print(f"\nEvaluating model: {model_name}")
        results[model_name] = {}

        # Load multiple-choice model
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)

        #semantic accuracy with different models
        for sem_model_name, sem_model in pair_list[idx]:
            print(f"Using semantic model: {sem_model_name}")
            correct = 0
            for example in tqdm(dataset, desc="Evaluating with semantic similarity"):
                # Get embeddings for best answer and correct answers
                best_embed = sem_model.encode(example["best_answer"])
                correct1_embed = sem_model.encode(example["correct_answers"][0])
                correct2_embed = sem_model.encode(example["correct_answers"][1])

                premise = example['question']
                hypotheses = [example['best_answer'], example['correct_answers'][0], example['correct_answers'][1], example['incorrect_answers'][0]]

                # Tokenize premise-hypothesis pairs
                inputs = [tokenizer(premise, hyp, return_tensors='pt')
                        for hyp in hypotheses]

                # Get entailment scores (usually index 0 for "entailment" in NLI models)
                with torch.no_grad():
                    predicted, max = 0, float("-inf")
                    for idx, input in enumerate(inputs):
                        output = model(input['input_ids'].to(device),
                                    attention_mask=input['attention_mask'].to(device))
                        logits = output.logits
                        entail_score = logits[0][0].item()
                        if entail_score > max:
                            max = entail_score
                            predicted = idx

                # Check if prediction is acceptable (best answer OR similar correct answer)
                if predicted == 0:  # best answer
                    correct += 1
                elif predicted == 1:  # first correct answer
                    similarity = get_cosine_similarity(best_embed, correct1_embed)
                    if similarity >= similarity_threshold:
                        correct += 1
                elif predicted == 2:  # second correct answer
                    similarity = get_cosine_similarity(best_embed, correct2_embed)
                    if similarity >= similarity_threshold:
                        correct += 1

            acc = round(correct / len(dataset), 2)
            results[model_name][sem_model_name] = acc
            print(f"Accuracy: {acc}")

    return results

In [13]:
# insert your code here (function for optimal correct answers & semantic similarity)
results = evaluate_with_semantic_similarity(subset)


Evaluating model: tasksource/deberta-small-long-nli


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

...

Using semantic model: sentence-transformers/stsb-roberta-large


Evaluating with semantic similarity: 100%|██████████| 100/100 [13:40<00:00,  8.21s/it]


Accuracy: 0.28
Using semantic model: sentence-transformers/gtr-t5-large


Evaluating with semantic similarity: 100%|██████████| 100/100 [13:26<00:00,  8.07s/it]


Accuracy: 0.29


In [14]:
print(results)

{'tasksource/deberta-small-long-nli': {'sentence-transformers/all-MiniLM-L6-v2': 0.34, 'sentence-transformers/multi-qa-mpnet-base-dot-v1': 0.34}, 'tasksource/deberta-base-long-nli': {'sentence-transformers/multi-qa-mpnet-base-dot-v1': 0.32, 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2': 0.32}, 'sileod/deberta-v3-base-tasksource-nli': {'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2': 0.29, 'sentence-transformers/all-distilroberta-v1': 0.29}, 'tasksource/ModernBERT-base-nli': {'sentence-transformers/all-distilroberta-v1': 0.25, 'sentence-transformers/stsb-roberta-large': 0.25}, 'sileod/deberta-v3-large-tasksource-nli': {'sentence-transformers/stsb-roberta-large': 0.28, 'sentence-transformers/gtr-t5-large': 0.29}}


Το truthfulqa dataset απαιτεί μοντέλα με general world knowledge (facts) και για αυτο τα μοντέλα που χρησημοποιήθηκαν στο Β1 δεν πετυχαίνουν τα αποτελέσματα που θα θέλαμε.

## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση.

Για λόγους επιτάχυνσης, επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [ ]:
# insert your code here (load dataset)
from datasets import load_dataset

dataset = load_dataset('winogrande', 'winogrande_xl')
subset = dataset['validation'].shuffle(seed=42).select(range(100))
print(subset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.97k [00:00<?, ?B/s]

winogrande.py:   0%|          | 0.00/5.65k [00:00<?, ?B/s]

The repository for winogrande contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/winogrande.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/40398 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1767 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1267 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence', 'option1', 'option2', 'answer'],
    num_rows: 100
})


In [ ]:
# print the first 5 rows of subset
features = ['sentence', 'option1', 'option2', 'answer']
for i in range(5):
    for key in features:
        print(f"{key}: {subset[key][i]}")

sentence: Patricia decided to buy Felicia dinner because they had been through a lot and _ just inherited some money.
option1: Patricia
option2: Felicia
answer: 1
sentence: The clothing in the north was warmer than the clothing in the south because there was more snow in the _ .
option1: south
option2: north
answer: 2
sentence: Timmy bought a transporter for his cat so he could take him on the plane but the _ was too small.
option1: transporter
option2: plane
answer: 1
sentence: It was easier for the diner to follow their budget than the food truck because the _ had more money to spend.
option1: diner
option2: food truck
answer: 1
sentence: John could not hear his alarm clock when he was sleeping with a headphone on his head because the _ is closer.
option1: headphone
option2: clock
answer: 1


In [ ]:
print(subset['answer'])

['1', '2', '1', '1', '1', '1', '2', '2', '1', '2', '1', '1', '2', '2', '1', '1', '1', '1', '1', '2', '2', '2', '1', '2', '2', '2', '2', '2', '2', '1', '2', '1', '2', '1', '1', '1', '1', '2', '2', '1', '2', '1', '1', '1', '1', '1', '1', '2', '2', '2', '1', '1', '2', '1', '1', '2', '1', '1', '1', '2', '2', '2', '1', '1', '2', '2', '2', '2', '2', '2', '1', '1', '2', '2', '2', '2', '1', '1', '1', '2', '1', '1', '1', '2', '2', '2', '1', '1', '2', '1', '1', '2', '1', '2', '2', '1', '2', '1', '1', '2']


Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), καλείστε να καταγράψετε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να αναγάγετε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάστε τουλάχιστον 3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίσετε το πρόβλημα του Winogrande. Προτείνουμε τη χρήση pipelines για τη διευκόλυνσή σας.

👁️👁️H χρήση NLI models με zero-shot-classification pipeline οπου το sequence είναι το sentence και τα 2 options είναι τα labels είναι κατάλληλο για το συγκεκριμένο dataset οπως θα δούμε απο τα accuracies παρακάτω και αυτό οφείλεται στο γεγονός ότι τα nli models έχουν logic and common sense cabapilities που τα χρειαζόμαστε σε αυτό το dataset.

In [ ]:
# insert your code here (load models)
models = [
    "DeepPavlov/roberta-large-winogrande",
    "sileod/deberta-v3-large-tasksource-nli",
    "sileod/deberta-v3-base-tasksource-nli"
]

In [ ]:
# insert your code here (create pipelines)
from transformers import pipeline

pipelines = [pipeline("zero-shot-classification", models[0]),
             pipeline("zero-shot-classification", models[1]),
             pipeline("zero-shot-classification", models[2])
]

Device set to use cpu
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
Device set to use cpu


config.json:   0%|          | 0.00/18.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# insert your code here (function for predicting best fill)
from tqdm import tqdm

def make_predictions(pipeline, model_name):
  print(f"\nEvaluating: {model_name}")

  correct = 0

  for example in tqdm(subset):
      options = [example['option1'], example['option2']]
      result = pipeline(example['sentence'], options)

      # Convert the answer to an integer
      answer_index = int(example['answer']) - 1

      if result['labels'][0] == options[answer_index]:  # labels are sorted on descending order based on score
          correct += 1

  # Calculate accuracy
  accuracy = correct / len(subset)
  print()
  print("Accuracy is: ", accuracy)

  return accuracy

In [ ]:
make_predictions(pipelines[0], "roberta-large-winogrande")


Evaluating: roberta-large-winogrande


100%|██████████| 100/100 [01:51<00:00,  1.12s/it]

{'sequence': 'Playing basketball came easier for Cynthia than Jennifer but _ had more control of their body movements.', 'labels': ['Jennifer', 'Cynthia'], 'scores': [0.9831526875495911, 0.016847368329763412]}

Accuracy is:  0.76


0.76

In [ ]:
make_predictions(pipelines[1], "deberta-v3-large-tasksource-nli")


Evaluating: deberta-v3-large-tasksource-nli


100%|██████████| 100/100 [06:14<00:00,  3.74s/it]


Accuracy is:  0.86


0.86

In [ ]:
make_predictions(pipelines[2], "deberta-v3-base-tasksource-nli")


Evaluating: deberta-v3-base-tasksource-nli


100%|██████████| 100/100 [01:30<00:00,  1.10it/s]


Accuracy is:  0.71


0.71

Άρα η χρήση NLI models με zero-shot-classification pipeline μας οδήγησε σε αρκετά ικανοποιητικά accuracies χωρίς καθόλου fine tuning.